<a href="https://colab.research.google.com/github/anguslou/Web-scraper/blob/master/web_scraper_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://towardsdatascience.com/looking-for-a-house-build-a-web-scraper-to-help-you-5ab25badc83e

In [0]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [0]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

In [0]:
sapo = "https://casa.sapo.pt/Venda/Apartamentos/?sa=11&or=10"
# sapo = "https://www.indeed.hk/jobs?q=president&l=hong+kong"
response = get(sapo, headers=headers)

In [7]:
print(response)

<Response [200]>


In [10]:
print(response.text[:1000])


<!DOCTYPE html>

<html lang="pt-PT">
<head><title>
	Casas para Venda, Apartamentos Com Ofertas recentes no Distrito de Lisboa, CASA SAPO - Portal Nacional de Imobiliário
</title><meta name="author" content="CASA SAPO - Portal Nacional de Imobiliário - Janela Digital SA" />
    <meta name="application-name" content="CASA SAPO - Portal Nacional de Imobiliário" data-copyright="Janela Digital SA" data-generated-time="07/11/2019 08:59" />
    
<meta name="content-language" content="pt-PT" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<meta name="description" content="Casas para Venda, Apartamentos Com Ofertas recentes no Distrito de Lisboa, Casas para Venda, Deseja comprar casa? No maior Portal Imobiliário Nacional temos milhares de apartamentos e moradias em Lisboa, no Porto e por todo o país." />
<meta name="keywords" content="Casas para Venda, Apartamentos Com Ofertas recentes no Distrito de Lisboa, Casas para Venda, venda, compra, comprar, casa


In [0]:
html_soup = BeautifulSoup(response.text, 'html.parser')

In [0]:
#identify each result in page

house_containers = html_soup.find_all('div', class_="searchResultProperty")

In [13]:
first = house_containers[0]
first.find_all('span')

[<span>
                         Apartamento T2, Loures
                     </span>,
 <span class="btnContactPVPI" id="MC_PropertyInList_repProperties_btnContactPVPINormal_0" onclick="ShowContactForm('742384e1-0128-11ea-980d-060000000052', '13', '5', true); return false;" style="z-index: 9999;" title="Contacte Anunciante">Contacte Anunciante</span>,
 <span>425 000 <strong title="Euro">€</strong></span>]

In [14]:
#find 3rd 'span' string

var_1 = first.find_all('span')[2].text
var_1

'425\xa0000 €'

In [15]:
#replace special character by ''

var_1 = var_1.replace('\xa0','')
var_1

'425000 €'

In [16]:
# using itertools to retrieve the digits and turn it into a float
var_1 = int(''.join(itertools.takewhile(str.isdigit, var_1)))
print(var_1,type(var_1))

425000 <class 'int'>


In [18]:
# Location, understand each field before for loop
location = first.find_all('p', class_='searchPropertyLocation')[0].text
location[7:location.find(',')]

'               Loures\r\n               '

In [19]:
# size in m2
first.find_all('p')[7]

<p>184m²</p>

In [23]:
# date ad posted
pd.to_datetime(first.find_all('div', class_="searchPropertyDate")[0].text[21:31])

IndexError: ignored

In [24]:
# short description
first.find_all('p', class_="searchPropertyDescription"[0].text[7:-6])

AttributeError: ignored

In [25]:
# gets all the links
for url in first.find_all('a'):
  print(url.get('href'))

/comprar-apartamento-t2-loures-tem-jacuzzi,aspiracao-central,terraco,seguranca,ar-condicionado,varanda,garagem-estacionamento,aquecimento-central,varandas-742384e1-0128-11ea-980d-060000000052.html?fpn=1
/comprar-apartamento-t2-loures-tem-jacuzzi,aspiracao-central,terraco,seguranca,ar-condicionado,varanda,garagem-estacionamento,aquecimento-central,varandas-742384e1-0128-11ea-980d-060000000052.html?fpn=1
/comprar-apartamento-t2-loures-tem-jacuzzi,aspiracao-central,terraco,seguranca,ar-condicionado,varanda,garagem-estacionamento,aquecimento-central,varandas-742384e1-0128-11ea-980d-060000000052.html?fpn=1
/comprar-apartamento-t2-loures-tem-jacuzzi,aspiracao-central,terraco,seguranca,ar-condicionado,varanda,garagem-estacionamento,aquecimento-central,varandas-742384e1-0128-11ea-980d-060000000052.html?fpn=1


In [26]:
# adding the domain part to the url and remove last letters
'https://casa.sapo.pt/' + first.find_all('a')[0].get('href')[1:-6]

'https://casa.sapo.pt/comprar-apartamento-t2-loures-tem-jacuzzi,aspiracao-central,terraco,seguranca,ar-condicionado,varanda,garagem-estacionamento,aquecimento-central,varandas-742384e1-0128-11ea-980d-060000000052.html'

In [27]:
# why not get a thumbnail too? it seems to be somewhere between all these characters
p = str(first.find_all('img')[0])
p

'<img alt="Ver Apartamento T2 em Loures" data-error="/Themes/Global/Images/NoPic280x210.jpg" data-src="https://media.casasapo.pt/Z320x240/Wnone/S5/C5379/P14762733/Tphoto/IDed42e100-0000-0500-0000-000008761649.jpg" data-src-retina="https://media.casasapo.pt/Z640x480/Wnone/S5/C5379/P14762733/Tphoto/IDed42e100-0000-0500-0000-000008761649.jpg" id="MC_PropertyInList_repProperties_thumbImage_0" src="https://media.casasapo.pt/Z320x240/Wnone/S5/C5379/P14762733/Tphoto/IDed42e100-0000-0500-0000-000008761649.jpg">\n<a class="photoLayer" href="/comprar-apartamento-t2-loures-tem-jacuzzi,aspiracao-central,terraco,seguranca,ar-condicionado,varanda,garagem-estacionamento,aquecimento-central,varandas-742384e1-0128-11ea-980d-060000000052.html?fpn=1" id="MC_PropertyInList_repProperties_hlRealestateDetail_Image_0" title="Ver Apartamento T2 em Loures"></a>\n<div class="photoButtonsBackground"></div>\n</img>'

In [28]:
# i want the 2x size thumbnail, which can be found in the positions defined below
imgurl = p[p.find('data-original_2x=') + 18:p.find('id=')-2]
imgurl

'rtamento T2 em Loures" data-error="/Themes/Global/Images/NoPic280x210.jpg" data-src="https://media.casasapo.pt/Z320x240/Wnone/S5/C5379/P14762733/Tphoto/IDed42e100-0000-0500-0000-000008761649.jpg" data-src-retina="https://media.casasapo.pt/Z640x480/Wnone/S5/C5379/P14762733/Tphoto/IDed42e100-0000-0500-0000-000008761649.jpg'

In [0]:
# setting up the lists that will form our dataframe with all the results
titles = []
created = []
prices = []
areas = []
zone = []
condition = []
descriptions = []
urls = []
thumbnails = []

In [31]:

%%time

n_pages = 0

for page in range(0,900):
    n_pages += 1
    sapo_url = 'https://casa.sapo.pt/Venda/Apartamentos/?sa=11&lp=10000&or=10'+'&pn='+str(page)
    r = get(sapo_url, headers=headers)
    page_html = BeautifulSoup(r.text, 'html.parser')
    house_containers = page_html.find_all('div', class_="searchResultProperty")
    if house_containers != []:
        for container in house_containers:
            
            # Price            
            price = container.find_all('span')[2].text
            if price == 'Contacte Anunciante':
                price = container.find_all('span')[3].text
                if price.find('/') != -1:
                    price = price[0:price.find('/')-1]
            if price.find('/') != -1:
                price = price[0:price.find('/')-1]
            
            price_ = [int(price[s]) for s in range(0,len(price)) if price[s].isdigit()]
            price = ''
            for x in price_:
                price = price+str(x)
            prices.append(int(price))

            # Zone
            location = container.find_all('p', class_="searchPropertyLocation")[0].text
            location = location[7:location.find(',')]
            zone.append(location)

            # Title
            name = container.find_all('span')[0].text
            titles.append(name)

            # Status
            status = container.find_all('p')[5].text
            condition.append(status)

            # Area
            m2 = container.find_all('p')[9].text
            if m2 != '-':
                m2 = m2.replace('\xa0','')
                m2 = float("".join(itertools.takewhile(str.isdigit, m2)))
                areas.append(m2)
                
            else:
                m2 = container.find_all('p')[7].text
                if m2 != '-':
                    m2 = m2.replace('\xa0','')
                    m2 = float("".join(itertools.takewhile(str.isdigit, m2)))
                    areas.append(m2)
                else:
                    areas.append(m2)

            # Creation date
            date = pd.to_datetime(container.find_all('div', class_="searchPropertyDate")[0].text[21:31])
            created.append(date)

            # Description
            desc = container.find_all('p', class_="searchPropertyDescription")[0].text[7:-6]
            descriptions.append(desc)

            # url
            link = 'https://casa.sapo.pt/' + container.find_all('a')[0].get('href')[1:-6]
            urls.append(link)

            # image
            img = str(container.find_all('img')[0])
            img = img[img.find('data-original_2x=')+18:img.find('id=')-2]
            thumbnails.append(img)
    else:
        break
    
    sleep(randint(1,2))
    
print('You scraped {} pages containing {} properties.'.format(n_pages, len(titles)))

IndexError: ignored

In [0]:
cols = ['Title', 'Zone', 'Price', 'Size (m²)', 'Status', 'Description', 'Date', 'URL', 'Image']

lisboa = pd.DataFrame({'Title': titles,
                           'Price': prices,
                           'Size (m²)': areas,
                           'Zone': zone,
                           'Date': created,
                           'Status': condition,
                           'Description': descriptions,
                           'URL': urls,
                           'Image': thumbnails})[cols]

lisboa.to_excel('lisboa_raw.xls')

# lisboa = pd.read_excel('lisboa_raw.xls')